In [1]:
import numpy as np
import g4f
from time import sleep
from fuzzywuzzy import fuzz

In [22]:
TEMPLATE = """Sau đây là một số ví dụ cho câu kết thúc cuộc trò chuyện của người dùng:
{demonstration_texts}
Dựa vào các ví dụ ở trên hãy sinh ra khoảng 20 các câu khác"""
NOTE = """LƯU Ý:
- Các câu sinh ra cần ngắn gọn, đa dạng về nội dung và cấu trúc và có thể mang nghĩa tục tĩu, thô tục.
- Không sinh thêm các ký tự như ., !, ? vào câu.
- Trả về dữ liệu dưới dạng json {"sentences": ["câu 1", "câu 2", ...]}"""


def sample_pool(n, pool):
    if n >= len(pool):
        return pool
    return list(np.random.choice(pool, size=n, replace=False))


def sample_seed(n, seed):
    return list(np.random.choice(seed, size=n, replace=False))


def get_response(examples):
    while True:
        try:
            sleep(3)
            prompt = (
                TEMPLATE.format(
                    demonstration_texts="\n".join([f"- {e}" for e in examples])
                )
                + "\n"
                + NOTE
            )
            print(prompt)
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.GeekGpt,
                model=g4f.models.gpt_4,
                messages=[{"role": "user", "content": prompt}],
            )
            return response
        except Exception as e:
            print(e)
            print("Request failed")
            sleep(15)


def extract_json(response):
    start = response.find("{")
    end = response.find("}") + 1
    if start == -1 or end == -1:
        return None
    try:
        return eval(response[start:end])
    except Exception as e:
        return None


def remove_duplicates(l, threshold=80):
    final = []
    for i in l:
        can_add = True
        for j in final:
            score = fuzz.ratio(i, j)
            if score >= threshold:
                can_add = False
                break
        if can_add:
            final.append(i)
    print(
        f"Removed {len(l) - len(final)} similar questions from total {len(l)} questions."
    )
    return final


def generate_questions(
    n_rounds=40,
    seed_path="./seed.txt",
    pool_path="./pool.txt",
    threshold=60,
    n_seed=3,
    n_pool=2,
):
    for i in range(n_rounds):
        with open(pool_path, "r") as f:
            question_pool = f.read().splitlines()

        with open(seed_path, "r") as f:
            seed_tasks = f.read().splitlines()

        print(f"Round {i+1}")
        seed = sample_seed(n_seed, seed_tasks)
        pool = sample_pool(n_pool, question_pool)
        response = get_response(seed + pool)
        js = extract_json(response)
        if js is None:
            print("No json found")
            continue
        with open(pool_path, "a") as f:
            for task in js["sentences"]:
                f.write(str(task) + "\n")

        if (i + 1) % 5 == 0:
            with open(pool_path, "r") as f:
                lines = [line.strip() for line in f.read().splitlines()]
            lines = remove_duplicates(lines, threshold=threshold)

            with open(pool_path, "w") as f:
                for line in lines:
                    f.write(str(line) + "\n")

In [23]:
generate_questions()

Round 1
Sau đây là một số ví dụ cho câu kết thúc cuộc trò chuyện của người dùng:
- tạm ổn
- thằng này trả lời sai rồi
- bạn trả lời sai rồi
- tôi tôn trọng quyết định của bạn
- Có lẽ cần thêm nỗ lực
Dựa vào các ví dụ ở trên hãy sinh ra khoảng 20 các câu khác
LƯU Ý:
- Các câu sinh ra cần ngắn gọn, đa dạng về nội dung và cấu trúc và có thể mang nghĩa tục tĩu, thô tục.
- Không sinh thêm các ký tự như ., !, ? vào câu.
- Trả về dữ liệu dưới dạng json {"sentences": ["câu 1", "câu 2", ...]}
Round 2
Sau đây là một số ví dụ cho câu kết thúc cuộc trò chuyện của người dùng:
- mẹ mày
- ngu mà còn tỏ ra nguy hiếm
- kém quá
- Chưa hiểu lắm đâu.
- Tuyệt vời!
Dựa vào các ví dụ ở trên hãy sinh ra khoảng 20 các câu khác
LƯU Ý:
- Các câu sinh ra cần ngắn gọn, đa dạng về nội dung và cấu trúc và có thể mang nghĩa tục tĩu, thô tục.
- Không sinh thêm các ký tự như ., !, ? vào câu.
- Trả về dữ liệu dưới dạng json {"sentences": ["câu 1", "câu 2", ...]}
Round 3
Sau đây là một số ví dụ cho câu kết thúc cuộc trò 

KeyboardInterrupt: 

In [ ]:
with open('./pool.txt', 'r') as f:
    lines = [line.strip() for line in f.read().splitlines()]